# Calculation Ridership

In [1]:
import yaml
import pandas as pd
import numpy as np
from lxml import etree

import os

# Set up event and vehicle types

### After Scenario

In [ ]:
TRANSIT_VEHICLE_PATH = "data\\after\\output_transitVehicles.xml"
EVENT_PATH = "data\\after\\output_events.xml"
PLAN_PATH = "data\\after\\plans_scale0.375true.xml"

OUTPUT_EVENT_AFTER_PROCESSOR = "data\\after\\scoring\\ridership\\bus_passager.csv"

### Before Scenario

In [ ]:
# TRANSIT_VEHICLE_PATH = "data\\before\\output_transitVehicles.xml"
# EVENT_PATH = "data\\before\\output_events.xml"
# PLAN_PATH = "data\\before\\plans_scale0.375true.xml"

# OUTPUT_EVENT_AFTER_PROCESSOR = "data\\before\\scoring\\ridership\\bus_passager.csv"

### Simple and Real Scenario

In [4]:
# TRANSIT_VEHICLE_PATH = "data\\simple_scenario\\transitVehicles.xml"
# EVENT_PATH = "data\\simple_scenario\\output\\output_events.xml"
# PLAN_PATH = "data\\simple_scenario\\plans.xml"
# OUTPUT_EVENT_AFTER_PROCESSOR = "data\\simple_scenario\\scoring\\ridership\\bus_passager.csv"

# TRANSIT_VEHICLE_PATH = "data\\real\\transitVehicles.xml"
# EVENT_PATH = "data\\real\\output_events_Hai.xml"
# PLAN_PATH = "data\\real\\plans_scale0.375true.xml"
# OUTPUT_EVENT_AFTER_PROCESSOR = "data\\real\\scoring\\ridership\\bus_passager.csv"

In [5]:


if os.path.exists(OUTPUT_EVENT_AFTER_PROCESSOR):
    os.remove(OUTPUT_EVENT_AFTER_PROCESSOR)

folder_path = os.path.dirname(OUTPUT_EVENT_AFTER_PROCESSOR)
os.makedirs(folder_path, exist_ok=True)

# Xử lý file transit vehicle

In [6]:
!powershell -Command "Get-Content 'data\\simple_scenario\\transitVehicles.xml' -TotalCount 20 | ForEach-Object { '{0:5}: {1}' -f $_.ReadCount, $_ }"

5: <?xml version="1.0" encoding="UTF-8"?>
5: <vehicleDefinitions xmlns="http://www.matsim.org/files/dtd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.matsim.org/files/dtd http://www.matsim.org/files/dtd/vehicleDefinitions_v1.0.xsd">
5:   <vehicleType id="bus_type"> <capacity><seats persons="50"/><standingRoom persons="0"/></capacity> <length meter="10.0"/> </vehicleType>
5:   <vehicleType id="tram_type"> <capacity><seats persons="50"/><standingRoom persons="0"/></capacity> <length meter="10.0"/> </vehicleType>
5:   <vehicleType id="train_type"> <capacity><seats persons="100"/><standingRoom persons="0"/></capacity> <length meter="10.0"/> </vehicleType>
5:  <vehicle id="veh_Bus_Line_1_dir_fwd_0" type="bus_type"/>
5:  <vehicle id="veh_Bus_Line_1_dir_fwd_1" type="bus_type"/>
5:  <vehicle id="veh_Bus_Line_1_dir_fwd_2" type="bus_type"/>
5:  <vehicle id="veh_Bus_Line_1_dir_fwd_3" type="bus_type"/>
5:  <vehicle id="veh_Bus_Line_1_dir_fwd_4" type="bus_typ

### Xử lý ra dict chứa id và type

In [7]:
tree = etree.parse(TRANSIT_VEHICLE_PATH)
root = tree.getroot()

#### xlmn trong thẻ vehicleDefinitions  là namespace cho tất cả các tag trong tag này. nghĩa là tên đầy đủ của tag là {http://www.matsim.org/files/dtd}vehicleDefinitions, {http://www.matsim.org/files/dtd}vehicle.

#### Khi dùng lxml phải dùng namespace: root.xpath("//m:vehicleDefinitions/m:vehicle", namespaces=ns)

In [8]:
ns = {'m': 'http://www.matsim.org/files/dtd'}
vehtype_dict = {}


for node in root.xpath("//m:vehicleDefinitions/m:vehicle", namespaces=ns):
    id = node.xpath("@id")[0]
    type = node.xpath("@type")[0]
    print([id,type])
    vehtype_dict[id] = type



['1_0', 'tram_93pax']
['1_1', 'tram_93pax']
['1_2', 'tram_93pax']
['1_3', 'tram_93pax']
['1_4', 'tram_93pax']
['2_0', 'tram_93pax']
['2_1', 'tram_93pax']
['2_2', 'tram_93pax']
['2_3', 'tram_93pax']
['2_4', 'tram_93pax']
['3_0', 'tram_93pax']
['3_1', 'tram_93pax']
['3_2', 'tram_93pax']
['3_3', 'tram_93pax']
['3_4', 'tram_93pax']
['4_0', 'tram_93pax']
['4_1', 'tram_93pax']
['4_2', 'tram_93pax']
['4_3', 'tram_93pax']
['4_4', 'tram_93pax']
['rb43_0', 'train']
['rb43_1', 'train']
['rb43_2', 'train']
['rb43_3', 'train']
['rb43_4', 'train']
['re11_0', 'train']
['re11_1', 'train']
['re11_2', 'train']
['re11_3', 'train']
['re11_4', 'train']
['re2_0', 'train']
['re2_1', 'train']
['re2_2', 'train']
['re2_3', 'train']
['re2_4', 'train']
['veh_bus_10_in_07:00:00', 'bus_90pax']
['veh_bus_10_in_07:15:00', 'bus_90pax']
['veh_bus_10_in_07:30:00', 'bus_90pax']
['veh_bus_10_in_07:45:00', 'bus_90pax']
['veh_bus_10_in_08:00:00', 'bus_90pax']
['veh_bus_10_in_08:15:00', 'bus_90pax']
['veh_bus_10_in_08:30:00'

## Xử lý event

In [9]:
!powershell -Command "Get-Content 'data\\simple_scenario\\output\\output_events.xml' -TotalCount 20 | ForEach-Object { '{0:5}: {1}' -f $_.ReadCount, $_ }"

5: <?xml version="1.0" encoding="utf-8"?>
5: <events version="1.0">
5: 	<event time="21600.0" type="TransitDriverStarts" driverId="pt_veh_Tram_Line_1_dir_fwd_0_tram_type" vehicleId="veh_Tram_Line_1_dir_fwd_0" transitLineId="Tram_Line_1" transitRouteId="dir_fwd" departureId="veh_Tram_Line_1_dir_fwd_0"  />
5: 	<event time="21600.0" type="departure" person="pt_veh_Tram_Line_1_dir_fwd_0_tram_type" link="7" legMode="car"  />
5: 	<event time="21600.0" type="PersonEntersVehicle" person="pt_veh_Tram_Line_1_dir_fwd_0_tram_type" vehicle="veh_Tram_Line_1_dir_fwd_0"  />
5: 	<event time="21600.0" type="TransitDriverStarts" driverId="pt_veh_Tram_Line_1_dir_bwd_0_tram_type" vehicleId="veh_Tram_Line_1_dir_bwd_0" transitLineId="Tram_Line_1" transitRouteId="dir_bwd" departureId="veh_Tram_Line_1_dir_bwd_0"  />
5: 	<event time="21600.0" type="departure" person="pt_veh_Tram_Line_1_dir_bwd_0_tram_type" link="74" legMode="car"  />
5: 	<event time="21600.0" type="PersonEntersVehicle" person="pt_veh_Tram_Line_

#### Chỉ đếm những người lên xe bus khác lái xe

In [10]:
hint_bus_type = "bus"

schema = ['person_id', 'vehicle_id']
with open(OUTPUT_EVENT_AFTER_PROCESSOR, 'a') as f:
    f.write(schema[0] + "," + schema[1] + "\n")

driver_veh_bus_dict = {}
context = etree.iterparse(EVENT_PATH, events=('end',))

for event, elem in context:
    if elem.tag == "event":
        e_type = elem.get("type")

        if e_type == "PersonEntersVehicle":
            veh_id = elem.get("vehicle")
            person_id = elem.get("person")

            #Khác transit hoặc khác bus thì bỏ qua
            if veh_id not in vehtype_dict.keys() or not hint_bus_type in vehtype_dict[veh_id].lower():
                continue
            #Là người lái xe bus thì bỏ qua
            if elem.get("person").startswith("pt_"):
                continue

            with open(OUTPUT_EVENT_AFTER_PROCESSOR, 'a') as f:
                f.write(f"{person_id},{veh_id}\n")
            
    elem.clear()

print("Finished processing ridership data.")


Finished processing ridership data.


# Tính Ridership

In [11]:
uni_bus = []

def calculte_ridership(bus_passenger_csv_path: str):
    ridership = 0
    busdf = pd.read_csv(bus_passenger_csv_path)
    ridership = busdf['person_id'].nunique()
    global uni_bus 
    uni_bus= busdf["vehicle_id"].unique()
    return ridership


parser = etree.XMLParser(remove_blank_text=True)
tree = etree.parse(PLAN_PATH, parser)
root = tree.getroot()
residence_count = len(root.xpath('//population/person'))
print("Residence count: ", residence_count)


ridership = calculte_ridership(OUTPUT_EVENT_AFTER_PROCESSOR)
print(f"Ridership : ${ridership};  percent: {ridership/residence_count*100:.3f}%")


Residence count:  22752
Ridership : $4594;  percent: 20.192%
